In [1]:
%run Return_Dataframes.ipynb
%run Functions.ipynb

In [2]:
import dash
from dash.dependencies import Input, Output, State
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import plotly.graph_objs as go

In [3]:
import re
import pandas as pd
pd.set_option('mode.chained_assignment', None)

In [4]:
from jupyter_plotly_dash import JupyterDash

### Dashboard Initialization

In [5]:
colors = {"background": "#111111", "text": "#A4B2AB", "plot" : "#45464A", "main_background" : "#111111"}
tab_style = {'borderBottom': '1px solid #d6d6d6','padding': '6px'}
tab_selected_style = {'borderTop': '1px solid #d6d6d6','borderBottom': '1px solid #d6d6d6','padding': '6px','fontWeight': 'bold'}

### Load Data

In [6]:
sentiment_data_on_load = return_sentiment_df()
unique_sentiment_search_terms = return_unique_searched_terms(sentiment_data_on_load['sentiment_id'])

In [7]:
sentiment_data_on_load.shape

(1239, 6)

In [8]:
graph_data_on_load = return_graph_df()
unique_graph_search_terms = return_unique_searched_terms(graph_data_on_load['graph_id'])

In [65]:
graph_data_on_load

,graph_id,id,text,screen_name,user_id,followers_count,user_mentions_screen_name,user_mentions_id,retweeted_screen_name,retweeted_id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,search_term
0,music_graphtable_1573897956.287716,1195640783531106304,RT @sapphictays: THE WAY EVERY SINGLE MAN IN T...,cliomckeever,758089000792064005,149,sapphictays,1121641030762127360,sapphictays,1121641030762127360,None,None,None,music
1,music_graphtable_1573897956.495385,1195640784277688320,RT @sinful_lola: Fela has had a huge influence...,tobi0525,886820746672185345,332,sinful_lola,1086051116322881537,sinful_lola,1086051116322881537,None,None,None,music
2,music_graphtable_1573897956.539135,1195640784491614208,RT @allkpop: [User Content] Kim Jaejoong wins ...,rasheee36308187,1168370028175081472,55,allkpop,21366823,allkpop,21366823,None,None,None,music
3,music_graphtable_1573897956.623359,1195640784902610944,RT @MasterMind_33: POWERFUL MUSIC \n11 DAY ( F...,JassiJones_11,75077679,1181,MasterMind_33,1188821643168231424,MasterMind_33,1188821643168231424,None,None,None,music
4,music_graphtable_1573897956.718935,1195640785409998848,RT @GOT7CBinfo: GOT7 is #5 today on Music Core...,markbamie,2954209891,440,GOT7CBinfo,1085206388123459585,GOT7CBinfo,1085206388123459585,None,None,None,music
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57400,sports_graphtable_1574049919.594459,1196278162646130688,RT @NFL: The @Raiders even up the score. #Raid...,1upJuan,56027421,186,NFL,19426551,NFL,19426551,None,None,None,sports
57401,sports_graphtable_1574049922.094325,1196278173136113664,RT @Neo_Edo_Exican: OC #tittytop #nsfw\nDoing ...,G_lahndi,967001382287941634,532,Neo_Edo_Exican,939611491623243776,Neo_Edo_Exican,939611491623243776,None,None,None,sports
57402,sports_graphtable_1574049922.685865,1196278175577387009,RT @SInow: Confirmed: Lamar Jackson is the mos...,anthony05419860,1176858150236037120,25,SInow,28370738,SInow,28370738,None,None,None,sports
57403,sports_graphtable_1574049931.059334,1196278210566197249,RT @CoachZachSmith: I sincerely believe @joelk...,_Stubs_,628150840,684,CoachZachSmith,48352979,CoachZachSmith,48352979,None,None,None,sports


### Graphs

In [10]:
def create_sentiment_map(sentiment_df, unique_sentiment_search_terms, sentiment_term):
    
    if sentiment_term is None:
        sentiment_term = [unique_sentiment_search_terms[0]]
    
    sentiment_df['search_term'] = sentiment_df.apply (lambda row: row['sentiment_id'].split('_', 1)[0], axis=1)
    
    dff = sentiment_df[sentiment_df['search_term'].isin(sentiment_term)]
    
    dff['colour'] = dff.apply (lambda row: sentiment_colors(row['sentiment']), axis=1)
    
    dff['text'] = 'User: ' + dff['user_name'] + '<br>Tweet: ' + dff['tweet'] + '<br>Sentiment: ' + dff['sentiment']
    
    sentiment_title = [i.title() for i in sentiment_term]
    sentiment_title_join = ", ".join(sentiment_title)
    
    return {
        'data': [go.Scattergeo(
            lon=dff['longitude'],
            lat=dff['latitude'],
            mode='markers',
            marker_color = dff['colour'],
            text= dff['text']
        )],
    
        'layout': 
        go.Layout(
            title= str('Sentiment Analysis for ' + sentiment_title_join),
            plot_bgcolor = colors['plot'],
            paper_bgcolor = colors['plot'],
            font = dict(color = colors['text']),
            geo = dict(
                showland=True, showocean=True, 
                showcountries=True, 
#                 projection = dict(
#                     type='equirectangular'
#                 )
            )

        )
    }

In [11]:
def create_sentiment_weightage(sentiment_df, unique_sentiment_search_terms, sentiment_term = None):
    
    if sentiment_term is None:
        sentiment_term = [unique_sentiment_search_terms[0]]
    
    sentiment_df['search_term'] = sentiment_df.apply (lambda row: row['sentiment_id'].split('_', 1)[0], axis=1)
    
    dff = sentiment_df[sentiment_df['search_term'].isin(sentiment_term)]
  
    labels = ['Positive', 'Neutral', 'Negative']
    colors_sentiment = ["#00ff00", "#ffff00", "ff0000"]
    
    values = []
    for sentiment in labels:
        count = dff[dff['sentiment']==sentiment]['sentiment'].count()
        values.append(count)
        
    sentiment_title = [i.title() for i in sentiment_term]
    sentiment_title_join = ", ".join(sentiment_title)
    
    return {
        "data": [go.Pie(labels=labels, values=values, hole=.5, marker_colors=colors_sentiment)],
        "layout": go.Layout(
            title= str("Sentiment Weightage for "+ sentiment_title_join),

            showlegend=True,
            legend=go.layout.Legend(
                orientation ="h"
            ),
                  
            plot_bgcolor = colors['plot'],
            paper_bgcolor = colors['plot'],
            font = dict(color = colors['text'])
        )
    }    

In [12]:
def create_interaction_graph(graph_df, unique_graph_search_terms, graph_term = None):
    
    if graph_term is None:
        graph_term = [unique_graph_search_terms[0]]
    
    graph_df['search_term'] = graph_df.apply (lambda row: row['graph_id'].split('_', 1)[0], axis=1)
    
    dff = graph_df[graph_df['search_term'].isin(graph_term)]
    
    graph_title = [i.title() for i in graph_term]
    graph_title_join = ", ".join(graph_title)
        
    G = create_network_graph(dff)
    
    edge_trace = go.Scatter(
        x=[],
        y=[],
        mode='lines'
    )
        
    for edge in G.edges():
        x0, y0 = G.node[edge[0]]['pos']
        x1, y1 = G.node[edge[1]]['pos']
        edge_trace['x'] += tuple([x0, x1, None])
        edge_trace['y'] += tuple([y0, y1, None])
        
    node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='RdBu',
        reversescale=True,
        color=[],
        size=15,
        colorbar=dict(
            thickness=10,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=0))
    )
    
    for node in G.nodes():
        x, y = G.node[node]['pos']
        node_trace['x'] += tuple([x])
        node_trace['y'] += tuple([y])
        
    for node, adjacencies in enumerate(G.adjacency()):
        node_trace['marker']['color']+=tuple([len(adjacencies[1])])
        node_info = adjacencies[0] +' # of connections: '+str(len(adjacencies[1]))
        node_trace['text']+=tuple([node_info])
        
    return {
        "data": [edge_trace, node_trace],
        "layout": go.Layout(
            title= str("Interaction Network for "+ graph_title_join),                 
            plot_bgcolor = colors['plot'],
            paper_bgcolor = colors['plot'],
            showlegend=False,
            font = dict(color = colors['text']),
            annotations=[ dict(
                    text="No. of connections",
                    showarrow=False,
                    xref="paper", yref="paper") ],
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        )
    } 

In [60]:
graph_df = graph_data_on_load
graph_df['search_term'] = graph_df.apply(lambda row: row['graph_id'].split('_', 1)[0], axis=1)
dff = graph_df[graph_df['search_term'].isin(unique_graph_search_terms)]
G = create_network_graph(dff)

In [63]:
graph = G

In [64]:
pr = nx.pagerank(graph)
pr

{'902390965599473664': 0.5, '19426551': 0.5}

In [ ]:
pr = nx.pagerank(graph)
sorted_nodes = sorted([(node, pagerank) for node, pagerank in pr.items()], key=lambda x:pr[x[0]])

api = tweepy.API(autorize_twitter_api(), wait_on_rate_limit_notify=True)
users = api.lookup_users(user_ids=[pair[0] for pair in sorted_nodes[:top_n_users]])

list_users = []
for u in users:
    list_users.append(u.screen_name)

df = pd.DataFrame(list_users, columns=["Top User Ranking"])

In [62]:
return_top_n_users_page_rank(G)

,Top User Ranking
0,bangkitriambada
1,NFL


In [13]:
def create_page_rank_table(graph_df, unique_graph_search_terms, graph_term = None):

    graph_df = graph_data_on_load

    if graph_term is None:
        graph_term = [unique_graph_search_terms[0]]

    graph_df['search_term'] = graph_df.apply(lambda row: row['graph_id'].split('_', 1)[0], axis=1)

    dff = graph_df[graph_df['search_term'].isin(graph_term)]

    graph_title = [i.title() for i in graph_term]
    graph_title_join = ", ".join(graph_title)

    G = create_network_graph(dff)

    users = return_top_n_users_page_rank(G)

    return {
            "data": [go.Table(
                header=dict(
                        values=['Top Users'],
                        line_color=colors['plot'],
                        font = dict(color = colors['text'], size=30), 
                        align='center',
                        height=50
                        ),
                cells=dict(
                    values=[list(users['Top User Ranking'])],
                    line_color=colors['plot'],

                    font = dict(color = colors['text'], size=30), 
                    align='center',
                    height=50
                
                ))

            ],
            "layout": go.Layout(
                title= str("Top User Ranking for "+ graph_title_join),
                plot_bgcolor = colors['plot'],
                paper_bgcolor = colors['plot'],
                font = dict(color = colors['text']),
  
                
            )
        }    


### App Initialization

In [48]:
app_name = "Twitter Analysis"
app = JupyterDash(name=app_name)

In [49]:
app.layout = html.Div(
    style={"backgroundColor": colors["main_background"]},
    children=[
        ##Header
        html.Div([
            html.Header(
                html.H1(
                    "Twitter Analyzer",
                    style={
                        "padding": "1.5rem",
                        "align-items": "center",
                        "color": colors["text"],
                        "textAlign": "center",
                    },
                )
            )],className="row",
        ),
                
        ##Tabs
        html.Div([
            dcc.Tabs(
                id="tabs",
            
                children=[

  
                    ##Tab 1     
                    dcc.Tab(
                        label="Analysis",
                        style = tab_style,
                        selected_style = tab_selected_style,
                        children=[
                        
            
                    ]),

                    ##Tab 2     
                    dcc.Tab(
                        label="Tweet Sentiment",
                        style = tab_style,
                        selected_style = tab_selected_style,
                        children=[
                        
                          html.Div([
                             html.Br(),
                               html.Div([
                                    html.Label(
                                        "Terms:",
                                        style={
                                            "background-color": colors["background"],
                                            "color": colors["text"],
                                        },
                                    ),
                                    dcc.Dropdown(
                                        id="sentiment_terms_picker",
                                        options=[
                                            {
                                                "label": id.title(),
                                                "value": id,
                                            }
                                            for id in unique_sentiment_search_terms
                                        ],
                                        clearable=False,
                                        multi=True,
                                        style={"width": "250px"},
                                        value=unique_sentiment_search_terms[0]
    
                                    ),
                                    ],style={ "display": "inline-block","padding-left": "0.5rem", "background-color": colors["background"],"color": colors["text"]},
                                ),
                              
                            html.Div([
                                html.Br(),
                                html.Div([
                                    dcc.Graph(
                                        id="sentiment_map",
                                        figure=create_sentiment_map(sentiment_data_on_load, 
                                                                    unique_sentiment_search_terms, 
                                                                    [unique_sentiment_search_terms[0]]),
                                        style ={
                                            "padding": "0.5rem",
                                            
#                                             "height": "100vh",
                                        },
                                    )
                                    ],style={"display": "inline-block", 'width': '60%'},
                                ), 
                                html.Div([
                                    dcc.Graph(
                                        id="sentiment_weightage",
                                        figure=create_sentiment_weightage(sentiment_data_on_load, 
                                                                          unique_sentiment_search_terms,
                                                                          [unique_sentiment_search_terms[0]]),
                                        style ={
                                            "padding": "0.5rem",
#                                             "height": "100vh",
                                        },
                                    )
                                    ],style={"display": "inline-block", 'width': '40%'},
                                ), 
                            ])     
                         ]),
            
                    ]),
                            
                    ##Tab 3     
                    dcc.Tab(
                        label="Twitter Graph",
                        style = tab_style,
                        selected_style = tab_selected_style,
                        children=[
                        
                 html.Div([
                             html.Br(),
                               html.Div([
                                    html.Label(
                                        "Terms:",
                                        style={
                                            "background-color": colors["background"],
                                            "color": colors["text"],
                                        },
                                    ),
                                    dcc.Dropdown(
                                        id="graph_terms_picker",
                                        options=[
                                            {
                                                "label": id.title(),
                                                "value": id,
                                            }
                                            for id in unique_graph_search_terms
                                        ],
                                        clearable=False,
                                        multi=True,
                                        style={"width": "250px"},
                                        value=unique_graph_search_terms[0]
    
                                    ),
                                    ],style={ "display": "inline-block","padding-left": "0.5rem", 
                                             "background-color": colors["background"],"color": colors["text"]},
                                ),
                              
                            html.Div([
                                html.Br(),
                                html.Div([
                                    dcc.Graph(
                                        id="interaction_map",
                                        figure=create_interaction_graph(graph_data_on_load, 
                                                                        unique_graph_search_terms,
                                                                       [unique_graph_search_terms[0]]),
                                        style ={
                                            "padding": "0.5rem",
#                                             "height": "100vh",
                                        },
                                    )
                                    ],style={"display": "inline-block", 'width': '60%'},
                                ), 
                                html.Div([
                                dcc.Graph(
                                        id="user_rank_table",
                                        figure=create_page_rank_table(graph_data_on_load, 
                                                                      unique_graph_search_terms,
                                                                     [unique_graph_search_terms[0]]),
                                        style ={
                                            "padding": "0.5rem",
#                                             "height": "100vh",
                                        },
                                    )
                                    ],style={"display": "inline-block", 'width': '40%'},
                                ),             

                            ])     
                         ]),
            
                    ]),
        
 
                ],   style ={
#                     'height': '44px'
                }),
            ]),
        ])

In [50]:
# @app.callback(Output('sentiment_map', 'figure'),
#               [Input('sentiment_terms_picker', 'value')])
# def update_sentiment_map(sentiment_term):

#     figure = create_sentiment_map(sentiment_data_on_load, unique_sentiment_search_terms, sentiment_term)
        
#     return figure

In [51]:
# @app.callback(Output('sentiment_weightage', 'figure'),
#               [Input('sentiment_terms_picker', 'value')])
# def update_sentiment_weightage(sentiment_term):
    
#     figure = create_sentiment_weightage(sentiment_data_on_load, unique_sentiment_search_terms, sentiment_term)
    
#     return figure

In [52]:
@app.callback(Output('interaction_map', 'figure'),
              [Input('graph_terms_picker', 'value')])
def update_interaction_map(graph_term):

    figure = create_interaction_graph(graph_data_on_load, unique_graph_search_terms, graph_term)
    
    return figure

In [53]:
# @app.callback(Output('user_rank_table', 'figure'),
#               [Input('graph_terms_picker', 'value')])
# def update_user_rank_table(graph_term):
   
#     figure = create_page_rank_table(graph_data_on_load, unique_graph_search_terms)
    
#     return figure

### Render App

In [54]:
app